In [1]:
import numpy as np
import os

In [2]:
ann_dict = np.load('./image_ann_dict.npz')['im_name2ann'][()]

In [3]:
def build_explanation_pair(explanation_info_pos, rm_word_list, im_name_new):
    '''
    Return (explanation_info_pos, explanation_info_neg) if flipped
    otherwise return (None, None)
    '''
    
    # The input has to be ground-truth (labeled True)
    if not explanation_info_pos['label']:
        return None, None
    
    # A deep copy of explanation_info_pos
    explanation_info_neg = explanation_info_pos.copy()
    explanation_info_neg['noun_phrases'] = [{'phrase': p['phrase']} for p in explanation_info_pos['noun_phrases']]
    
    explanation_sentence = explanation_info_neg['explanation'].lower()
    
    is_sentence_flipped = False
    for p in explanation_info_neg['noun_phrases']:
        phrase = p['phrase'].lower()
        is_p_flipped = False
        for w in rm_word_list:
            if w in phrase:
                is_p_flipped = True
                break
        
        if is_p_flipped:
            try:
                assert(phrase in explanation_sentence)
                p['phrase'] = None
                explanation_sentence = explanation_sentence.replace(phrase, '')
                is_sentence_flipped = is_sentence_flipped | is_p_flipped
            except:
                print('phrase not found:')
                print(phrase)
                print(explanation_sentence)
    
    explanation_info_neg['explanation'] = explanation_sentence
    explanation_info_neg['noun_phrases'] = [{'phrase': p['phrase']} for p in explanation_info_neg['noun_phrases']
                                            if p['phrase'] is not None]
    if is_sentence_flipped:
        # exchange the pos and neg, since the flipped expression (removing 'head' or 'tail')
        # is the correct one 
        explanation_info_pos, explanation_info_neg = explanation_info_neg, explanation_info_pos
        explanation_info_pos['label'] = True
        explanation_info_pos['im_name'] = im_name_new
        explanation_info_neg['label'] = False
        explanation_info_neg['im_name'] = im_name_new
        
        return explanation_info_pos, explanation_info_neg
        
    return None, None

In [5]:
# Look at one example
im_name = '007.Parakeet_Auklet/Parakeet_Auklet_0001_795972.jpg'
ann = ann_dict[im_name]

explanation_info_pos, explanation_info_neg = build_explanation_pair(ann[0], ['belly'], '.')
print(explanation_info_pos)
print(explanation_info_neg)

{'explanation': 'a medium sized bird that has a  and a very short stout bill', 'im_name': '.', 'noun_phrases': [{'phrase': 'sized bird'}, {'phrase': 'short stout bill'}], 'label': True}
{'explanation': 'A medium sized bird that has a white belly and a very short stout bill', 'im_name': '.', 'noun_phrases': [{'phrase': 'sized bird'}, {'phrase': 'white belly'}, {'phrase': 'short stout bill'}], 'label': False}


In [6]:
ann_new_dict = {}

In [7]:
# chop head
path_prefix = 'remove_head/chop_head_'
rm_word_list = ['head', 'eye', 'eyes']
for im_name in ann_dict:
    ann = ann_dict[im_name]
    ann_new = []
    im_name_new = path_prefix + os.path.basename(im_name)
    for e in ann:
        explanation_info_pos, explanation_info_neg = build_explanation_pair(e, rm_word_list, im_name_new)
        if explanation_info_pos and explanation_info_neg:
            ann_new.append(explanation_info_pos)
            ann_new.append(explanation_info_neg)
        
    ann_new_dict[im_name_new] = ann_new
    
# chop tail
path_prefix = 'remove_body/chop_body_'
rm_word_list = ['tail']
for im_name in ann_dict:
    ann = ann_dict[im_name]
    ann_new = []
    im_name_new = path_prefix + os.path.basename(im_name)
    for e in ann:
        explanation_info_pos, explanation_info_neg = build_explanation_pair(e, rm_word_list, im_name_new)
        if explanation_info_pos and explanation_info_neg:
            ann_new.append(explanation_info_pos)
            ann_new.append(explanation_info_neg)
        
    ann_new_dict[im_name_new] = ann_new

phrase not found:
white eyes and feather
this is a black bird with an orange bill, white eyes, and feather coming out of head.
phrase not found:
black head short
the bird is small with an orange and black head, short pointed bill, white belly and small tail.
phrase not found:
speckled head throat and breast
this small bird has a brown speckled head, throat and breast, a white belly, brown and white wings and tail, grey thighs, and pink tarsus and feet.
phrase not found:
white striped back head and neck
a bird with a black and white striped back, head, and neck with brown sides.
phrase not found:
blue head orange chest
a very colorful bird with a blue head, orange chest, and wings that are yellow, blue, and green.
phrase not found:
crown eye brow
this bird is somewhat blurry but appears to be yellow all over except some black stripes on the crown, eye brow and wings.
phrase not found:
gray head breast and belly
this is a brown bird with a gray head, breast and belly and a pointed beak
p

In [8]:
np.savez('./image_ann_dict_remove_head_body.npz', im_name2ann=ann_new_dict)